## For European options, option value from binomial model fluctuates around the value derived from black scholes model, thus, there is not really any advantage of using binomial model for european options.

## Binomial model is easy to understand, helps to analyze the working of the algorithm step by step and a good tool for teaching. 

## Real advantage of Binomial model is in pricing on those options whose value depends upon the path like barrier options or asian options, options with arbitrary payoffs at time T and ofcourse American and Bermudan options

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [4]:
%run ../Black-scholes-model-greeks-plotted/BSM-formula-implemented.ipynb

In [6]:
def get_binomial_parameters(sigma, r, dt):
    """
    sigma corresponds to real world annualized vol.
    meaning roughly underlying will increase or decrease by this much percent in a year
    so, for any dt, volatility will be sigma*sqrt(dt)
    """
    up = np.exp(sigma*np.sqrt(dt))
    down = np.exp(-sigma*np.sqrt(dt)) # down can also be written as 1 / up
    """S(t+dt) = exp(-rdt)*expunder risk neutral probability of St"""
    risk_neutral_conversion_factor = (np.exp(r*dt) - down) / (up - down)
    # above probability is computed assuming martingale property
    return up, down, risk_neutral_conversion_factor

In [94]:
def crr_value(S0, K, T, r, sigma, otype, M=4):
    """
    Implementing cox-ross-rubinstein (binomial_model)
    """
    dt = T / M
    df = np.exp(-r*dt)
    
    up, down, risk_neutral_conversion_factor = get_binomial_parameters(sigma, r, dt)
    
    # matrix creation
    mu = np.arange(M+1)
    mu = np.resize(mu, (M + 1, M + 1))
    md = np.transpose(mu)
    mu = u ** (mu - md)
    md = d ** md
    S = S0 * mu * md
    
    if otype == "call":
        V = np.maximum(S - K, 0)
    else:
        V = np.maximum(K - S, 0) # for put
    
    z = 0
    # backward iteration
    for t in range(M - 1, -1, -1):
        V[0 : M - z, t] = (risk_neutral_conversion_factor * V[0 : M - z, t + 1]
                          + (1 - risk_neutral_conversion_factor) * V[1:M - z + 1, t+1])*df
        z+=1
            
    return V[0,0]

In [95]:
S0 = 100.0  # index level
K = 100.0  # option strike
T = 1.0  # maturity date
r = 0.05  # risk-less short rate
sigma = 0.2  # volatility

In [96]:
crr_value(S0, K, T, r, sigma, "call")

9.970522921901265